In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 前処理

In [94]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import datetime

In [95]:
tweet_df=pd.read_csv('./tweet.csv')
#重複削除
tweet_df.drop_duplicates(subset='id', keep='first',inplace=True)

user_df=pd.read_csv('./user.csv')

#重複削除
user_df.drop_duplicates(subset='id', keep='first',inplace=True)

all_info_df=tweet_df.merge(user_df,how='inner',left_on='author_id',right_on='id')

In [96]:
#列名変更
all_info_df.rename(columns={'created_at_x': 'tweet_time', 'id_x': 'tweet_id', 'created_at_y': 'make_account_time', 'id_y': 'use_id'}, inplace=True)

In [97]:
all_info_df['tweet_time'].head()

0    2021-12-07T15:29:56.000Z
1    2021-12-07T15:14:45.000Z
2    2021-12-07T08:35:43.000Z
3    2021-12-07T08:34:01.000Z
4    2021-12-04T12:31:54.000Z
Name: tweet_time, dtype: object

In [98]:
#日本時間に変換。日本時間が昼か夜かもポイントになりそうなため。
all_info_df['tweet_time']=pd.DatetimeIndex(all_info_df['tweet_time']).tz_convert(None) + datetime.timedelta(hours=9)
all_info_df['make_account_time']=pd.DatetimeIndex(all_info_df['make_account_time']).tz_convert(None)  + datetime.timedelta(hours=9)

In [99]:
all_info_df['tweet_hour'] = all_info_df['tweet_time'].apply(lambda x: x.hour)

In [100]:
all_info_df.head()

,author_id,tweet_time,tweet_id,text,like_count,quote_count,reply_count,retweet_count,make_account_time,use_id,name,username,followers_count,following_count,listed_count,tweet_count,tweet_hour
0,934335139802267648,2021-12-08 00:29:56,1468241373463826432,RT @usa_usa0427: 忘年会はじまるよ〜〜！！！✨\n#ゼロイチファミリア\n#...,0,0,0,213,2017-11-25 17:17:11,934335139802267648,sesa騎士団(函館支部),sait36938371Ses,41,36,5,6991,0
1,934335139802267648,2021-12-08 00:14:45,1468237552591196175,RT @dwangojpnews: “2021年おつかれさま” #ゼロイチファミリア 総勢7...,0,0,0,166,2017-11-25 17:17:11,934335139802267648,sesa騎士団(函館支部),sait36938371Ses,41,36,5,6991,0
2,934335139802267648,2021-12-07 17:35:43,1468137132745445376,RT @akanekogurasi: 久しぶりのゼロイチ忘年会🥰\n同じテーブルです可愛いで...,0,0,0,135,2017-11-25 17:17:11,934335139802267648,sesa騎士団(函館支部),sait36938371Ses,41,36,5,6991,17
3,934335139802267648,2021-12-07 17:34:01,1468136704976760835,RT @H_yume_1018: 忘年会始まるよ〜！！！\n\n#ゼロイチファミリア\n#ゼ...,0,0,0,200,2017-11-25 17:17:11,934335139802267648,sesa騎士団(函館支部),sait36938371Ses,41,36,5,6991,17
4,934335139802267648,2021-12-04 21:31:54,1467109404160163844,RT @miyu_kishi0213: 忘年会ライブSOLD OUTありがとうございます🥲🥲...,0,0,0,24,2017-11-25 17:17:11,934335139802267648,sesa騎士団(函館支部),sait36938371Ses,41,36,5,6991,21


In [101]:
all_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119306 entries, 0 to 119305
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   author_id          119306 non-null  int64         
 1   tweet_time         119306 non-null  datetime64[ns]
 2   tweet_id           119306 non-null  int64         
 3   text               119306 non-null  object        
 4   like_count         119306 non-null  int64         
 5   quote_count        119306 non-null  int64         
 6   reply_count        119306 non-null  int64         
 7   retweet_count      119306 non-null  int64         
 8   make_account_time  119306 non-null  datetime64[ns]
 9   use_id             119306 non-null  int64         
 10  name               119298 non-null  object        
 11  username           119306 non-null  object        
 12  followers_count    119306 non-null  int64         
 13  following_count    119306 non-null  int64   

In [102]:
use_columns=[
             'text',
             'tweet_hour',
             'like_count',
             'followers_count',
             ]

num_features = [
             'tweet_hour',
             'followers_count',
             ]
    

In [103]:
use_tweet_df=all_info_df[use_columns].copy()

In [104]:
use_tweet_df.head()

,text,tweet_hour,like_count,followers_count
0,RT @usa_usa0427: 忘年会はじまるよ〜〜！！！✨\n#ゼロイチファミリア\n#...,0,0,41
1,RT @dwangojpnews: “2021年おつかれさま” #ゼロイチファミリア 総勢7...,0,0,41
2,RT @akanekogurasi: 久しぶりのゼロイチ忘年会🥰\n同じテーブルです可愛いで...,17,0,41
3,RT @H_yume_1018: 忘年会始まるよ〜！！！\n\n#ゼロイチファミリア\n#ゼ...,17,0,41
4,RT @miyu_kishi0213: 忘年会ライブSOLD OUTありがとうございます🥲🥲...,21,0,41


In [105]:
#標準化
import pickle
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
use_tweet_df[num_features] = sc.fit_transform(use_tweet_df[num_features])
with open('./standardscaler_for_tweetlikepred.pkl', mode='wb') as f:
    pickle.dump(sc,f)


In [106]:
sc.scale_

array([6.28183307e+00, 1.45235724e+04])

In [107]:
#標準化できてるっぽい
use_tweet_df.describe()

,tweet_hour,like_count,followers_count
count,1.193060e+05,119306.000000,1.193060e+05
mean,3.588507e-14,4.541649,1.482131e-16
std,1.000004e+00,106.076023,1.000004e+00
min,-2.490922e+00,0.000000,-9.388333e-02
25%,-5.806511e-01,0.000000,-8.899472e-02
50%,3.744841e-01,0.000000,-7.653222e-02
75%,8.520517e-01,1.000000,-4.320709e-02
max,1.170430e+00,27292.000000,1.038527e+02


In [108]:
use_tweet_df.to_csv('./preprocess_tweet_data.csv', index=False)